<a href="https://colab.research.google.com/github/LuFernandez/neural-networks/blob/master/naive_bayesV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes
## Redes Neuronales

© 2020 Lucero G. Fernandez



In [0]:
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

### Carga de datos


In [0]:
! wget "https://raw.githubusercontent.com/rn-2019-itba/Clase-2--Hiperparametros-y-Tecnicas-de-Validacion/master/Opcional/data/emails.csv"
clear_output()

In [0]:
dataset = pd.read_csv('emails.csv')

Se tienen 5728 emails de dos tipos: text y spam. En text están las palabras y en spam si es o no spam (1==spam)

In [0]:
#for i in range(len(dataset['spam'])):
#  if dataset.spam[i]==1:
 #   print(dataset.spam[i])

 #la cantidad de emails que son spam
 spam_count=np.sum(dataset.spam)

Probabilidad a priori de que sea spam:

In [0]:
print(spam_count/len(dataset)*100)

23.88268156424581


#Preprocesamiento de datos


In [0]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [0]:
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
stemmer = PorterStemmer()
clear_output()


##\#1 tokenization, lemmatization, stop-words, stemming

In [0]:
emails_raw=dataset.text
emails_filtered=list()

for n in range(len(dataset)):
  tok = word_tokenize(emails_raw[n])  #tokenization
  lem = [lemmatizer.lemmatize(x,pos = 'v') for x in tok] #lemmatization
  stop = [x for x in lem if x not in stopwords.words('english')] #stop words
  stem = [stemmer.stem(x) for x in stop] #stemming
  alpha = [x for x in stem if x.isalpha()]  #filter non words

  emails_filtered.append(" ".join(alpha))




Guardo en disco

In [0]:
import pickle

with open('em_filt.pck', 'wb') as fp:
    pickle.dump(emails_filtered, fp)

In [0]:
with open ('em_filt.pck', 'rb') as fp:
    itemlist = pickle.load(fp)

###TfidVectorizer


In [0]:
max_df=[.5, .6, .7, .75, .8, .85, .9]
min_df=[10, 20, 30, 40, 50, 75, 100, 250]
alpha=[.001, .01, .1, .5, 1, 2]

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB



#Count Vectorizer

# count_vect = CountVectorizer(max_df=.8,min_df=10)
# raw_data=count_vect.fit_transform(emails_filtered)
# raw_data.toarray()
 
chosen_alpha=0
chosen_maxdf=0
chosen_mindf=0
maximum_porc=0
for minimo in min_df:
  for maximo in max_df:

    #TfidVectorizer
    tfidf_vect = TfidfVectorizer(max_df=maximo,min_df=minimo)
    raw_data = tfidf_vect.fit_transform(itemlist)
    vocabulary=tfidf_vect.get_feature_names()

    #Separo en train, validation, y test
    X_train, X_testprima, y_train, y_testprima = train_test_split(raw_data, dataset['spam'], test_size=0.80)
    X_val, X_test, y_val, y_test = train_test_split(X_testprima, y_testprima, test_size=0.5)

    #entreno modelo
    for alfa in alpha:
      clf = MultinomialNB(alpha=alfa)
      clf.fit(X_train, y_train)
      #Analizo la precisión del modelo, con X_val e y_val
      porc=sum(np.array(clf.predict(X_val.toarray()))==np.array(y_val))/len(y_val)*100
      #print(porc)
      if porc>maximum_porc:
        maximum_porc=porc  #guardo valores
        chosen_alpha=alfa
        chosen_maxdf=maximo
        chosen_mindf=minimo


print(maximum_porc, chosen_alpha, chosen_maxdf, chosen_mindf)


98.77782627673504 0.1 0.5 10


###Entreno modelo de Bayes


In [0]:
#print(np.shape(dataset['text']))
#print(np.shape(X_train))
#print(np.shape(X_testprima))

#print(np.shape(dataset['spam']))
#print(np.shape(y_train))
#print(np.shape(y_testprima))